# Packages installation

In [14]:
!pip install datasets jsonlines > /dev/null
!git clone https://github.com/amazon-science/mxeval.git > /dev/null
!pip install -e mxeval > /dev/null
!touch /content/mxeval/__init__.py > /dev/null

fatal: destination path 'mxeval' already exists and is not an empty directory.


# Imports

In [15]:
import json
import re
import sys
import os

from datasets import load_dataset
import jsonlines
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    StoppingCriteria,
    StoppingCriteriaList,
)

from tqdm import tqdm

# Setup kotlinc


In [16]:
import sys
import os

# Define the file path
file_path = '/content/kotlin-compiler-2.0.20.zip'

# Check if the file already exists
if not os.path.exists(file_path):
    # If the file doesn't exist, download it
    !wget -O $file_path https://github.com/JetBrains/kotlin/releases/download/v2.0.20/kotlin-compiler-2.0.20.zip > /dev/null
    print(f"The file '{file_path}' has been downloaded.")
else:
    print(f"The file '{file_path}' already exists.")

The file '/content/kotlin-compiler-2.0.20.zip' already exists.


In [17]:
!sudo unzip kotlin-compiler-2.0.20.zip -d /usr/local/ > /dev/null

replace /usr/local/kotlinc/build.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [18]:
path_to_kotlinc = "/usr/local/kotlinc/bin"

current_path = os.environ.get('PATH')

path_list = current_path.split(os.pathsep)

if path_to_kotlinc in path_list:
    print(f"The path '{path_to_kotlinc}' ALREADY in the PATH.")
else:
    os.environ['PATH'] += ":" + path_to_kotlinc
    print(f"The path '{path_to_kotlinc}' ADDED to the PATH.")



The path '/usr/local/kotlinc/bin' ALREADY in the PATH.


In [19]:
!kotlinc -version

info: kotlinc-jvm 2.0.20 (JRE 11.0.24+8-post-Ubuntu-1ubuntu322.04)


## Test kotlinc

In [20]:
!echo 'fun main() { println("Hello, Kotlin 2.0.20 from Google Colab!") }' > hello.kt
!kotlinc hello.kt -include-runtime -d hello.jar
!java -jar hello.jar

Hello, Kotlin 2.0.20 from Google Colab!


# Setup mxeval package

In [21]:
# Get the path to the mxeval directory
current_dir = os.path.dirname('/content/')
mxeval_dir = os.path.join(current_dir, 'mxeval')

# Add mxeval to sys.path
if sys.path[0] != mxeval_dir:

    if mxeval_dir in sys.path:
        sys.path.remove(mxeval_dir)

    sys.path.insert(0, mxeval_dir)

# Print all system path for packages
print(sys.path, sep='\n')

['/content/mxeval', '/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor', '/root/.ipython']


# Imports mxeval

In [24]:
from mxeval.evaluation import evaluate_functional_correctness

# Code

In [4]:
class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops, tokenizer):
        (StoppingCriteria.__init__(self),)
        self.stops = rf"{stops}"
        self.tokenizer = tokenizer

    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
    ) -> bool:
        last_three_tokens = [int(x) for x in input_ids.data[0][-3:]]
        decoded_last_three_tokens = self.tokenizer.decode(last_three_tokens)

        return bool(re.search(self.stops, decoded_last_three_tokens))


def generate(problem):
    criterion = StoppingCriteriaSub(stops="\n}\n", tokenizer=tokenizer)
    stopping_criteria = StoppingCriteriaList([criterion])

    problem = tokenizer.encode(problem, return_tensors="pt").to('cuda')
    sample = model.generate(
        problem,
        max_new_tokens=256,
        min_new_tokens=128,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        num_beams=1,
        stopping_criteria=stopping_criteria,
    )

    answer = tokenizer.decode(sample[0], skip_special_tokens=True)
    return answer


def clean_asnwer(code):
    # Clean comments
    code_without_line_comments = re.sub(r"//.*", "", code)
    code_without_all_comments = re.sub(
        r"/\*.*?\*/", "", code_without_line_comments, flags=re.DOTALL
    )
    #Clean signatures
    lines = code.split("\n")
    for i, line in enumerate(lines):
        if line.startswith("fun "):
            return "\n".join(lines[i + 1:])

    return code


model_name = "JetBrains/CodeLlama-7B-Kexer"
dataset = load_dataset("jetbrains/Kotlin_HumanEval")['train']
problem_dict = {problem['task_id']: problem for problem in dataset}

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name)

output = []
for key in tqdm(list(problem_dict.keys()), leave=False):
    problem = problem_dict[key]["prompt"]
    answer = generate(problem)
    answer = clean_asnwer(answer)
    output.append({"task_id": key, "completion": answer, "language": "kotlin"})

output_file = f"answers"
with jsonlines.open(output_file, mode="w") as writer:
    for line in output:
        writer.write(line)

evaluate_functional_correctness(
    sample_file=output_file,
    k=[1],
    n_workers=16,
    timeout=15,
    problem_file=problem_dict,
)

with open(output_file + '_results.jsonl') as fp:
    total = 0
    correct = 0
    for line in fp:
        sample_res = json.loads(line)
        print(sample_res)
        total += 1
        correct += sample_res['passed']

print(f'Pass rate: {correct/total}')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.59k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/175k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/161 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

...

Running test suites...


100%|██████████| 161/161 [00:00<00:00, 977.34it/s] 


Writing results to answers_results.jsonl...


100%|██████████| 161/161 [00:00<00:00, 25032.73it/s]


{'task_id': 'HumanEval_kotlin/32', 'completion': '    val lPrime = l.toMutableList()\n\n    for (i in lPrime.indices) {\n        if (i % 3 == 0) {\n            lPrime[i] = lPrime[i].absoluteValue\n            lPrime[i] = lPrime[i].toChar().toInt()\n        }\n    }\n\n    return lPrime\n}\n', 'language': 'kotlin', 'result': "[Errno 2] No such file or directory: 'kotlinc'", 'passed': False, 'time_elapsed': None}
{'task_id': 'HumanEval_kotlin/74', 'completion': '    val b = Math.cbrt(a.toDouble()).toInt()\n    return a == b * b * b\n}\n', 'language': 'kotlin', 'result': "[Errno 2] No such file or directory: 'kotlinc'", 'passed': False, 'time_elapsed': None}
{'task_id': 'HumanEval_kotlin/160', 'completion': '    val evenDigits = mutableListOf<Int>()\n\n    if (a > b) {\n        return evenDigits\n    }\n\n    for (num in a..b) {\n        if (num % 2 == 0) {\n            evenDigits.add(num)\n        }\n    }\n\n    return evenDigits.sorted()\n}\n', 'language': 'kotlin', 'result': "[Errno 2

# Experements